### start

In [20]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
# ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.wallet_preprocessor as wp
import sagemaker_wallets.workflow_orchestrator as wo
import sage_utils.config_validation as ucv
import utils as u
from utils import ConfigError

# reload all modules
modules = [
    wm, wp, wo,
    u, ucv
]

from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#b45827"


# load all configs
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


u.export_code(
    code_directories=[
        'sagemaker_wallets',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
    output_file="temp/sagemaker_code.py"
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[15/Jul/25 21:59:50] INFO [utils.export_code:1667] Consolidation complete. All files are saved in temp/sagemaker_code.py
[15/Jul/25 21:59:50] MILESTONE [366077203.<module>:114] Good morning, let's get to work


# Initial ETLs

# Code begins

### Initiate orchestrator

In [21]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config)

### Load, preprocess, and upload data

#### load data

In [22]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffixes = [
    '231107',
    '240306'
]
workflow_orch.load_training_data(date_suffixes)

[15/Jul/25 21:59:53] MILESTONE [workflow_orchestrator.load_training_data:94] <DEV> Loading training data for 2 periods: ['231107', '240306']
[15/Jul/25 21:59:53] INFO [workflow_orchestrator.load_training_data:112] Training data loaded successfully: 8 splits, 16,000 total rows


##### inspect features

In [ ]:
# Create combined NaN count and describe statistics
nan_counts = workflow_orch.training_data['x_train'].isna().sum()
describe_stats = workflow_orch.training_data['x_train'].describe().T

# Combine into single DataFrame
combined_stats = pd.concat([
    nan_counts.rename('nan_count'),
    describe_stats
], axis=1).sort_index()

u.display_full(combined_stats.sort_index())

#### preprocess data

In [23]:
preprocessor = wp.SageWalletsPreprocessor(sage_wallets_config)
preprocessed_data = preprocessor.preprocess_training_data(workflow_orch.training_data)

[15/Jul/25 21:59:56] INFO [wallet_preprocessor.preprocess_training_data:42] Starting preprocessing for SageMaker XGBoost compatibility...
[15/Jul/25 21:59:56] INFO [wallet_preprocessor._handle_missing_values:165] Filled NaN values in 100 columns for x_train.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor._combine_x_y_data:259] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor.preprocess_training_data:77] Preprocessed train: 2,000 rows × 222 cols.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor._handle_missing_values:165] Filled NaN values in 100 columns for x_test.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor._combine_x_y_data:259] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor.preprocess_training_data:77] Preprocessed test: 2,000 rows × 222 cols.
[15/Jul/25 21:59:56] INFO [wallet_preprocessor._handle_missing_values:16

#### upload data

In [24]:
# Upload and retrieve URIs
workflow_orch.upload_training_data(preprocessed_data, overwrite_existing=True)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[15/Jul/25 21:59:59] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials
[15/Jul/25 21:59:59] INFO [workflow_orchestrator.upload_training_data:155] <DEV> Ready to upload 8 preprocessed training data files across 2 date folders.
[15/Jul/25 21:59:59] INFO [workflow_orchestrator.upload_training_data:157] Target variable: cw_crypto_net_gain_crypto_inflows_winsorized
[15/Jul/25 21:59:59] INFO [workflow_orchestrator.upload_training_data:158] Target: s3://wallet-training-data/training-data-preprocessed/dda-897_dev/[DATE]/
[15/Jul/25 22:00:01] INFO [workflow_orchestrator.upload_training_data:190] Uploading train_cw_crypto_net_gain_crypto_inflows_winsorized for 231107: 2,000 rows
[15/Jul/25 22:00:02] INFO [workflow_orchestrator.upload_training_data:199] Uploaded train to s3://wallet-training-data/training-data-preprocessed/dda-897_dev/231107/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv
[15/Jul/25 22:00:02] INFO [workflow_orchestrator.upload_tr

### Train Model

#### retrieve uris

In [25]:
date_suffixes = [
    '231107',
    '240306'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

{'231107': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/231107/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/231107/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/231107/eval_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'val': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/231107/val_cw_crypto_net_gain_crypto_inflows_winsorized.csv'},
 '240306': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/240306/train_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/240306/test_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-897_dev/240306/eval_cw_crypto_net_gain_crypto_inflows_winsorized.csv',
  'v

#### train model

In [28]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    s3_uris
)
date_suffix = date_suffixes[0]



In [29]:
modeling_results = modeler.train_model(date_suffix)

[15/Jul/25 22:04:49] INFO [wallet_modeler.train_model:79] Starting SageMaker training...
[15/Jul/25 22:04:49] INFO [image_uris._processor:530] Ignoring unnecessary instance type: None.
[15/Jul/25 22:04:53] INFO [wallet_modeler.train_model:159] Launching training job: wallet-xgb-dda-897-dev-231107-20250715-220453
[15/Jul/25 22:04:53] INFO [wallet_modeler.train_model:160] Model output path: s3://wallet-training-data/sagemaker-models/dda-897-dev/
[15/Jul/25 22:04:53] INFO [telemetry_logging.wrapper:91] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[15/Jul/25 22:04:53] INFO [session.submit:1053] Creating training-job with name: wallet-xgb-dda-897-dev-231107

2025-07-16 05:04:54 Starting - Starting the training job...
2025-07-16 05:05:28 Downloading - Downloading input data......
2025-07-16 05:06:13 Downloading - Downloading the training image...
2025-07-16 05:06:54 Training - Training image download completed. Training in progress..Arguments: train
[2025-07-16:05:06:57:INFO] Running standalone xgboost training.
[2025-07-16:05:06:57:INFO] File size need to be processed in the node: 6.94mb. Available memory size in the node: 348.44mb
[2025-07-16:05:06:57:INFO] Determined delimiter of CSV input is ','
[05:06:57] S3DistributionType set as FullyReplicated
[05:06:57] 2000x221 matrix with 442000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-07-16:05:06:57:INFO] Determined delimiter of CSV input is ','
[05:06:57] S3DistributionType set as FullyReplicated
[05:06:57] 2000x221 matrix with 442000 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[05:06:57] src/tree/update

[15/Jul/25 22:07:43] INFO [wallet_modeler.train_model:174] Training completed. Model stored at: s3://wallet-training-data/sagemaker-models/dda-897-dev/wallet-xgb-dda-897-dev-231107-20250715-220453/output/model.tar.gz


#### get existing model uri

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, s3_uris)
date_suffix = date_suffixes[0]


model = modeler.load_existing_model(date_suffix)

In [ ]:
predictions = modeler.predict_with_model()

### DDA 884 devspace